In [1]:
import pandas as pd
import plotly.express as px

hlasky = (pd
          .read_csv('../hlasky.csv')
          .fillna(value='NA', axis=1)
         )

### Kolik toho máme?

In [2]:
print('Celkový počet karet: {}'
      .format(len(hlasky))
)

print(
    'Počet rozpracovaných karet: {}'
    .format(len(hlasky.query('stav != "NA"')))
)

Celkový počet karet: 106
Počet rozpracovaných karet: 26


### Kde toho nejvíc chybí?

In [55]:
# simple plot do readme
stav_breakdown_simple = (hlasky
                         .replace('NA', 'Nezačato')
                         .replace(['clash', 'find', 'pick'], 'In progress')
                         .replace('done', 'Hotovo')
                         .groupby(['stav', 'kategorie'])
                         ['stav']
                         .count()
                         .to_frame()
                         .rename(columns={'stav': 'count'})
                         .reset_index()
                        )

simple_order = {'stav': ['Hotovo', 
                      "In progress", 
                      "Nezačato"]}

simple_colors = ["#99c140", "#e7b416", "#696969"]

fig = px.bar(
    stav_breakdown_simple,
    x='kategorie',
    y='count',
    color='stav',
    color_discrete_sequence=simple_colors,
    category_orders=simple_order,
    title='Hlášky ke kartám',
    labels = {'kategorie': 'Kategorie karty', 'count': 'Počet'}
)

fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    'yaxis.gridcolor': 'rgba(1, 1, 1, 0.1)',}
)

fig.write_image("stav.png")

In [64]:
# metadata pro nasledujici ploty
cat_order = {'stav': ['done', 
                      'pick', 
                      'find',
                      'clash',
                      'meh',
                      'NA']}

cat_colors = {
    'done': "#CECD7B",
    'pick': "#DAECED",
    'find': "#B5966D",
    'clash': "#EDCB64",
    'meh': "#B62A3D",
    'NA': "#A9A9A9"
}

In [65]:
# complex plot do notebooku
stav_breakdown = (hlasky
                  .groupby(['stav', 'kategorie'])
                  ['stav']
                  .count()
                  .to_frame()
                  .rename(columns={'stav': 'count'})
                  .reset_index()
                 )

fig = px.bar(
    stav_breakdown,
    x='kategorie',
    y='count',
    color='stav',
    color_discrete_map=cat_colors,
    category_orders = cat_order,
    title='Progress: hlášky (všechny karty)',
    labels = {'kategorie': 'Kategorie karty', 'count': 'Počet'}
)

fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    'yaxis.gridcolor': 'rgba(1, 1, 1, 0.1)',}
)

fig.show()

In [66]:
funkce_breakdown = (hlasky
                    .query('kategorie == ["hneda","modra","zelena"]')
                  .groupby(['stav', 'funkce'])
                  ['stav']
                  .count()
                  .to_frame()
                  .rename(columns={'stav': 'count'})
                  .reset_index()
                 )

fig = px.bar(
    funkce_breakdown,
    x='funkce',
    y='count',
    color='stav',
    color_discrete_map=cat_colors,
    category_orders=cat_order,
    title='Progress podle funkce (jen lízací karty)',
    labels = {'funkce': 'Funkce karty', 'count': 'Počet'}
)

fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    'yaxis.gridcolor': 'rgba(1, 1, 1, 0.1)',}
)

fig.show()

### Náhodná nedokončená karta

Co takhle pohnout s:

In [67]:
# display all columns
pd.set_option('display.max_columns', None)

# pick random unfinished card
random_card = hlasky.query('stav != "done"').sample()

# extract stav
current_stav = random_card['stav'].tolist()[0]

# make card name bold & stav colored
random_card.style.set_properties(
    subset=['stav'],
    **{'background-color': cat_colors[current_stav]}
).set_properties(
    subset=['karta'],
    **{'font-weight': 'bold'}
)

,pack,kategorie,funkce,karta,prepis,alternativy,stav,comment ke stavu
90,fistful,event,odhazovaci a lizaci se switchnou,Opusteny Dul,NA,NA,NA,NA
